In [4]:
# Clone the repository
!git clone https://github.com/ApoPeri/tensorgator.git
# Navigate to the directory
%cd tensorgator
# Install with the desired package name
!pip install -e . --install-option="--name=tensorgator"

c:\Users\hp\Desktop\code\tensorgator\examples\tensorgator


Cloning into 'tensorgator'...
C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///C:/Users/hp/Desktop/code/tensorgator/examples/tensorgator
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for tensorgator (pyproject.toml): started
  Building editable for tensorgator (pyproject.toml): finished with status 'done'
  Created wheel for tensorgator: filename=tensorgator-0.1.1-0.editable-py3-none-any.whl size=6922 sha256=0e04e331aacac6111b990a325946bcacd741a0687f6a0b55b400af839f9d5ffa
  Stored in directory: C:\Users\hp\AppData\Local\Temp\pip-ephem-wheel-cache-iktomw96\wheels\69\ce\ee\c6bdb9306dedc

DEPRECATION: --install-option is deprecated because it forces pip to use the 'setup.py install' command which is itself deprecated. pip 23.1 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11358
DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
#Change runtime (top left) by ram/disk to T4
!pip install numba

                                              0.0/2.7 MB ? eta -:--:--
     -----                                    0.4/2.7 MB 7.4 MB/s eta 0:00:01
     --------------------------               1.8/2.7 MB 16.3 MB/s eta 0:00:01
     ---------------------------------------- 2.7/2.7 MB 19.0 MB/s eta 0:00:00
                                              0.0/28.1 MB ? eta -:--:--
     --                                       2.0/28.1 MB 42.5 MB/s eta 0:00:01
     -----                                    4.2/28.1 MB 44.4 MB/s eta 0:00:01
     --------                                 5.7/28.1 MB 40.7 MB/s eta 0:00:01
     ----------                               7.7/28.1 MB 40.7 MB/s eta 0:00:01
     -------------                            9.2/28.1 MB 42.0 MB/s eta 0:00:01
     ---------------                         11.3/28.1 MB 43.7 MB/s eta 0:00:01
     ------------------                      13.5/28.1 MB 43.7 MB/s eta 0:00:01
     --------------------                    14.9/28.1 MB 40.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import time
import os
from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

# Import tensorgator
import tensorgator as tg
from tensorgator.prop_cuda import propagate_constellation_cuda

def create_earth_sphere(ax, radius=1.0, resolution=30):
    # Lower resolution Earth for faster rendering
    u = np.linspace(0, 2 * np.pi, resolution)
    v = np.linspace(0, np.pi, resolution)
    x = radius * np.outer(np.cos(u), np.sin(v))
    y = radius * np.outer(np.sin(u), np.sin(v))
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v))

    earth = ax.plot_surface(x, y, z, color='blue', alpha=1,
                           linewidth=0, antialiased=True)

    return earth

def main():
    print("Generating satellites in 3D space...")

    # Set random seed for reproducibility
    np.random.seed(21)

    RE = tg.RE

    num_sats = 4000
    constellation = []
    orbit_types = {
        'LEO': {'count': 1000, 'alt_range': (300000, 2000000), 'inc_range': (20, 98)},
        'MEO': {'count': 1000, 'alt_range': (5000000, 20000000), 'inc_range': (0, 90)},
        'GEO': {'count': 1000, 'alt_range': (35786000, 35786000), 'inc_range': (0, 5)},
        'HEO': {'count': 1000, 'alt_range': (500000, 40000000), 'inc_range': (60, 90), 'ecc_range': (0.2, 0.7)}
    }
    sat_categories = []

    # Generate satellites for each orbit type
    for orbit_type, params in orbit_types.items():
        count = params['count']
        for _ in range(count):
            # Random altitude within range
            alt_min, alt_max = params['alt_range']
            altitude = np.random.uniform(alt_min, alt_max)
            a = RE + altitude

            # Eccentricity (circular by default, except for HEO)
            if orbit_type == 'HEO':
                e_min, e_max = params['ecc_range']
                e = np.random.uniform(e_min, e_max)
            else:
                e = 0.0

            # Random inclination within range
            inc_min, inc_deg_max = params['inc_range']
            inc = np.radians(np.random.uniform(inc_min, inc_deg_max))

            # Random RAAN, argument of perigee, and mean anomaly
            raan = np.radians(np.random.uniform(0, 360))
            argp = np.radians(np.random.uniform(0, 360))
            M0 = np.radians(np.random.uniform(0, 360))

            constellation.append([a, e, inc, raan, argp, M0])
            sat_categories.append(orbit_type)

    constellation = np.array(constellation)

    # Time span (1 hour with 15-second steps for fewer frames)
    sim_duration = 3600
    time_step = 15  # Increased from 5 to 15
    times = np.arange(0, sim_duration, time_step)
    num_frames = len(times)

    print(f"Propagating {num_sats} satellites over {num_frames} time steps...")
    start_time = time.time()

    positions = propagate_constellation_cuda(constellation, times, return_frame='ecef')

    prop_time = time.time() - start_time
    print(f"Propagation completed in {prop_time:.2f} seconds")

    print("Creating 3D visualization...")

    fig = plt.figure(figsize=(8, 6), dpi=1000)
    ax = fig.add_subplot(111, projection='3d')

    earth_radius_scaled = 1.0
    create_earth_sphere(ax, radius=earth_radius_scaled)

    scale_factor = earth_radius_scaled / RE

    color_map = {
        'LEO': 'red',
        'MEO': 'green',
        'GEO': 'yellow',
        'HEO': 'magenta'
    }

    colors = [color_map[cat] for cat in sat_categories]

    x_init = positions[:, 0, 0] * scale_factor
    y_init = positions[:, 0, 1] * scale_factor
    z_init = positions[:, 0, 2] * scale_factor

    scatter = ax.scatter(x_init, y_init, z_init, s=1, alpha=0.3, c=colors)

    max_alt = np.max(constellation[:, 0]) * scale_factor
    ax.set_xlim(-max_alt, max_alt)
    ax.set_ylim(-max_alt, max_alt)
    ax.set_zlim(-max_alt, max_alt)

    ax.set_xlabel('X [ER]')
    ax.set_ylabel('Y [ER]')
    ax.set_zlabel('Z [ER]')
    ax.set_title('Satellites Orbiting Earth')

    ax.view_init(elev=20, azim=30)

    ax.text2D(0.05, 0.95, 'Red=LEO, Green=MEO, Yellow=GEO, Magenta=HEO',
             transform=ax.transAxes, fontsize=8)

    time_text = ax.text2D(0.05, 0.9, '', transform=ax.transAxes, fontsize=8)

    def update(frame):
        x = positions[:, frame, 0] * scale_factor
        y = positions[:, frame, 1] * scale_factor
        z = positions[:, frame, 2] * scale_factor
        scatter._offsets3d = (x, y, z)

        elapsed_minutes = frame * time_step / 60
        time_text.set_text(f'Time: {elapsed_minutes:.0f} min')

        return scatter, time_text

    print("Creating animation...")

    skip_frames = 1
    frames_to_render = range(0, num_frames, skip_frames)

    anim = FuncAnimation(fig, update, frames=frames_to_render, interval=50)

    print("Rendering movie (this may take a while)...")

    # Use ffmpeg for faster rendering if available
    try:
        # Try to use ffmpeg writer
        from matplotlib.animation import FFMpegWriter
        writer = FFMpegWriter(fps=24, metadata=dict(artist='Me'), bitrate=1800)
        anim.save('./satellites_movie.mp4', writer=writer)

        file_path = os.path.abspath('./satellites_movie.mp4')
        print(f"Full path to movie file: {file_path}")
    except Exception as e:
        # Fall back to pillow for GIF if ffmpeg not available
        print(f"FFmpeg error: {e}")
        print("Falling back to GIF format...")
        anim.save('./satellites_movie.gif', writer='pillow', fps=15)
        print("Movie saved as 'satellites_movie.gif'")

    plt.close()

if __name__ == "__main__":
    main()

Generating satellites in 3D space...
Propagating 4000 satellites over 240 time steps...
Propagation completed in 4.19 seconds
Creating 3D visualization...
Creating animation...
Rendering movie (this may take a while)...
Full path to movie file: c:\Users\hp\Desktop\code\tensorgator\examples\satellites_movie.mp4
